# A Comparison of mannitol and glucose as sole carbon sources

### Setup imports

In [1]:
import cobra
import pandas as pd
import sys

# Disable scientific notation
pd.set_option('display.float_format', '{:.6f}'.format)

source_dir = '../src'
sys.path.append(source_dir)
# from add_column_to_13c_flux_df import add_column_to_13c_flux_df
from flux_prediction_scatterplot import flux_prediction_scatterplot

### Load model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr_3.json")
model

Name,model
Memory address,112085150
Number of metabolites,1121
Number of reactions,1350
Number of genes,648
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Add farnesyl diphosphate (FPP) exchange reaction to create a terpenoid reaction to maximize

In [3]:
# create the exchange reaction for farnesyl diphosphate
exchange_reaction = cobra.Reaction('EX_frdp[c]')
exchange_reaction.name = 'Exchange reaction for farnesyl diphosphate'
exchange_reaction.subsystem = 'Exchange'
exchange_reaction.lower_bound = 0
exchange_reaction.upper_bound = 1000
exchange_reaction.add_metabolites({model.metabolites.get_by_id('frdp[c]'): -1})
model.add_reactions([exchange_reaction])

### Add mannitol uptake metabolites and reactions to the model

In [4]:
# Extracellular mannitol
mannitol_e = cobra.Metabolite(
    'mannitol[e]',
    formula='C6H14O6',
    name='Mannitol',
    compartment='C_e')

# Cytosolic mannitol
mannitol_c = cobra.Metabolite(
    'mannitol[c]',
    formula='C6H14O6',
    name='Mannitol',
    compartment='C_c')

mannitol_exchange = cobra.Reaction('EX_mannitol_e')
mannitol_exchange.name = 'mannitol exchange'
mannitol_exchange.lower_bound = 0.  
mannitol_exchange.upper_bound = 1000.  
mannitol_exchange.add_metabolites({
    mannitol_e: -1.0,
})

mannitol_transport = cobra.Reaction('mannitol_transport')
mannitol_transport.name = 'mannitol transport'
mannitol_transport.lower_bound = 0. 
mannitol_transport.upper_bound = 1000. 
mannitol_transport.add_metabolites({
    mannitol_e: -1.0,
    mannitol_c:  1.0,
}) 

# EC 1.14.13.244
# add this reaction: nad_c + mnl_c ⇌ fru_c + h_c + nadh_c
mannitol_dehydrogenase = cobra.Reaction('MNLDH2')
mannitol_dehydrogenase.name = 'mannitol dehydrogenase'
mannitol_dehydrogenase.lower_bound = 0.
mannitol_dehydrogenase.upper_bound = 1000.  # This is the default
mannitol_dehydrogenase.add_metabolites({
    mannitol_c: -1.0,
    model.metabolites.get_by_id('nadp[c]'):   -1.0,
    model.metabolites.get_by_id('h[c]'): 1.0,
    model.metabolites.get_by_id('fru[c]'): 1.0,
    model.metabolites.get_by_id('nadph[c]'):  1.0,
})

model.add_reactions([mannitol_exchange])
model.add_reactions([mannitol_transport])
model.add_reactions([mannitol_dehydrogenase])

### Run biomass-maximizing FBA for glucose

In [5]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_glc(e)': 100.0, # 100 mmol of glucose
  'EX_mannitol_e': 0,
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}
  
with model:
    # update the model to use the specified medium
    medium = model.medium

    # print the medium
    print('Media composition:')
    [print(model.medium[m], m) for m in model.medium]

    # set the objective to be the biomass reaction (this is the default behavior, but we'll be explicit)
    model.objective = model.reactions.get_by_id('EX_biom')

    # find the optimal solution
    solution = cobra.flux_analysis.pfba(model)

    print('\nBiomass flux', solution.fluxes['EX_biom'])

    # make a list of dictionaries with the reaction id, name, flux, and absolute flux
    reactions = []
    for reaction_id, flux in solution.fluxes.items():
      reactions.append({
        'reaction_id': reaction_id,
        'reaction_name': model.reactions.get_by_id(reaction_id).name,
        'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
        'glucose_max_biomass_flux': flux,
        'absolute_flux': abs(flux), # use for sorting, then drop
      })

    # make a dataframe from the list of dictionaries
    fba_results_df = pd.DataFrame(reactions)

    # sort the dataframe by absolute flux
    fba_results_df = fba_results_df.sort_values(by=['absolute_flux'], ascending=False)

    # drop the absolute flux column
    fba_results_df = fba_results_df.drop(columns=['absolute_flux'])

fba_results_df.head()


EX_mannitol_e does not seem to be an an exchange reaction. Applying bounds anyway.


Media composition:
100.0 EX_glc(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)

Biomass flux 11.398166174413957


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198


### Run biomass-maximizing FBA for mannitol

In [6]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_glc(e)': 0, 
  'EX_mannitol_e': 100.0, # 100 mmol of mannitol
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}
  
with model:
    # update the model to use the specified medium
    medium = model.medium

    # print the medium
    print('Media composition:')
    [print(model.medium[m], m) for m in model.medium]

    # set the objective to be the biomass reaction (this is the default behavior, but we'll be explicit)
    model.objective = model.reactions.get_by_id('EX_biom')

    # find the optimal solution
    solution = cobra.flux_analysis.pfba(model)

    print('\nBiomass flux', solution.fluxes['EX_biom'])
    
    # create a list to hold the max fpp fluxes
    flux_values = []

    # loop over the reactions in the fba_results_df dataframe, and add a column for FPP maximmizing glucose fluxes
    for _, row in fba_results_df.iterrows():
        reaction_id = row['reaction_id']
        flux = solution.fluxes[reaction_id]
        flux_values.append(flux)

# add the max fpp fluxes to the dataframe
fba_results_df['mannitol_max_biomass_flux'] = flux_values

fba_results_df.head()

EX_mannitol_e does not seem to be an an exchange reaction. Applying bounds anyway.


Media composition:
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)

Biomass flux 12.348013355615235


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307,-608.649499
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652,370.077123
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108,363.432451
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474,353.744597
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,402.370631


### Run FPP-maximizing FBA for glucose

In [7]:
# minimal medium with glucose as sole carbon source
model.medium = {
    'EX_glc(e)': 100.0, # 100 mmol of glucose
    'EX_mannitol_e': 0,
    'EX_h2o(e)': 10000.0,
    'EX_h(e)': 10000.0,
    'EX_nh4(e)': 10000.0,
    'EX_o2(e)': 10000.0,
    'EX_pi(e)': 10000.0,
    'EX_so4(e)': 10000.0,
}

solution = 0
  
with model:
    # update the model to use the specified medium
    medium = model.medium

    # print the medium
    print('Media composition:')
    [print(model.medium[m], m) for m in model.medium]

    # set the objective to be the farnesyl diphosphate exchange reaction
    model.objective = model.reactions.get_by_id('EX_frdp[c]')

    # find and display the optimal solution
    solution = cobra.flux_analysis.pfba(model)

    print('\nFPP flux', solution.fluxes['EX_frdp[c]'])

    # create a list to hold the max fpp fluxes
    max_fpp_fluxes = []

    # loop over the reactions in the fba_results_df dataframe, and add a column for FPP maximmizing glucose fluxes
    for _, row in fba_results_df.iterrows():
        reaction_id = row['reaction_id']
        flux = solution.fluxes[reaction_id]
        max_fpp_fluxes.append(flux)

# add the max fpp fluxes to the dataframe
fba_results_df['glucose_max_fpp_flux'] = max_fpp_fluxes

fba_results_df.head()

EX_mannitol_e does not seem to be an an exchange reaction. Applying bounds anyway.


Media composition:
100.0 EX_glc(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)

FPP flux 22.35123367198844


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,glucose_max_fpp_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307,-608.649499,-541.848089
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652,370.077123,281.761006
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108,363.432451,281.761006
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474,353.744597,281.761006
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,402.370631,354.136430


### Run FPP-maximizing FBA for mannitol

In [8]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_glc(e)': 0, 
  'EX_mannitol_e': 100.0, # 100 mmol of mannitol
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}

solution = 0
  
with model:
    # update the model to use the specified medium
    medium = model.medium

    # print the medium
    print('Media composition:')
    [print(model.medium[m], m) for m in model.medium]

    # set the objective to be the farnesyl diphosphate exchange reaction
    model.objective = model.reactions.get_by_id('EX_frdp[c]')

    # find and display the optimal solution
    solution = cobra.flux_analysis.pfba(model)

    print('\nFPP flux', solution.fluxes['EX_frdp[c]'])

    # create a list to hold the max fpp fluxes
    max_fpp_fluxes = []

    # loop over the reactions in the fba_results_df dataframe, and add a column for FPP maximmizing mannitol fluxes
    for _, row in fba_results_df.iterrows():
        reaction_id = row['reaction_id']
        flux = solution.fluxes[reaction_id]
        max_fpp_fluxes.append(flux)

# add the max fpp fluxes to the dataframe
fba_results_df['mannitol_max_fpp_flux'] = max_fpp_fluxes

fba_results_df.head()

EX_mannitol_e does not seem to be an an exchange reaction. Applying bounds anyway.


Media composition:
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)

FPP flux 23.58490566037736


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,glucose_max_fpp_flux,mannitol_max_fpp_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307,-608.649499,-541.848089,-623.270440
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652,370.077123,281.761006,326.100629
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108,363.432451,281.761006,326.100629
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474,353.744597,281.761006,326.100629
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,402.370631,354.136430,440.566038


### Analyze the fluxes out of the cell

In [9]:
# analyze the fate of carbon from mannitol
out_flux_df = fba_results_df[fba_results_df['reaction_id'].str.contains('EX_') == True]

# remove rows where glucose_max_biomass_flux	glucose_max_fpp_flux	mannitol_max_biomass_flux	and mannitol_max_fpp_flux are zero
out_flux_df = out_flux_df[(out_flux_df[['glucose_max_biomass_flux', 'glucose_max_fpp_flux', 'mannitol_max_biomass_flux', 'mannitol_max_fpp_flux']] != 0).all(axis=1)]

out_flux_df

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,glucose_max_fpp_flux,mannitol_max_fpp_flux
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,402.370631,354.136430,440.566038
391,EX_co2(e),CO2 exchange,co2[e] -->,167.905908,131.898067,264.731495,246.226415
444,EX_o2(e),O2 exchange,o2[e] <=>,-149.890948,-162.381860,-130.624093,-154.716981
442,EX_nh4(e),Ammonia exchange,nh4[e] <=>,-77.722667,-84.199556,-0.000000,-0.000000
451,EX_pi(e),Phosphate exchange,pi[e] <=>,-3.853221,-4.174323,-44.702467,-47.169811
422,EX_h(e),H exchange,h[e] <=>,-3.853221,-4.174323,-44.702467,-47.169811


### Define a function to get all the reactions with a certain metabolite

In [49]:
def get_reactions_with_metabolite(fba_results_df, metabolite_id):
    rows_to_add = []

    for _, row in fba_results_df.iterrows():
        reaction_id = row['reaction_id']
        reaction = model.reactions.get_by_id(reaction_id)
        metabolites = reaction.metabolites
        if any(met.id == metabolite_id for met in metabolites):
            rows_to_add.append(row)

    if rows_to_add:
        metabolite_df = pd.concat(rows_to_add, axis=1).T

        columns_to_check = ['glucose_max_biomass_flux', 'glucose_max_fpp_flux', 'mannitol_max_biomass_flux', 'mannitol_max_fpp_flux']
        metabolite_df = metabolite_df[(metabolite_df[columns_to_check].abs() > 0.001).any(axis=1)]

        
        return metabolite_df
    else:
        return pd.DataFrame()


### Analyze the source of extracellular CO2

In [50]:
get_reactions_with_metabolite(fba_results_df, 'co2[e]')

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,glucose_max_fpp_flux,mannitol_max_fpp_flux
957,CO2t,CO2 transporter via diffusion,co2[e] <=> co2[c],-167.905908,-131.898067,-264.731495,-246.226415
391,EX_co2(e),CO2 exchange,co2[e] -->,167.905908,131.898067,264.731495,246.226415


### Analyze sources of intracellular CO2

In [53]:
get_reactions_with_metabolite(fba_results_df, 'co2[c]')

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,glucose_max_fpp_flux,mannitol_max_fpp_flux
957,CO2t,CO2 transporter via diffusion,co2[e] <=> co2[c],-167.905908,-131.898067,-264.731495,-246.226415
999,CO2tm,CO2 transport diffusion mitochondrial,co2[c] <=> co2[m],-108.680635,-117.737355,-197.677794,-175.471698
703,HCO3E,HCO3 equilibration reaction,co2[c] + h2o[c] <=> h[c] + hco3[c],56.224379,60.909744,132.559265,125.157233
646,GND,phosphogluconate dehydrogenase,6pgc[c] + nadp[c] --> co2[c] + nadph[c] + ru5p...,53.430654,7.883208,0.000000,0.000000
1264,OMCDC,2 Oxo 4 methyl 3 carboxypentanoate decarboxyla...,3c4mop[c] + h[c] --> 4mop[c] + co2[c],15.169007,16.433091,66.279632,62.578616
1339,LEUDH,2-Oxoisovalerate dehydrogenase,4mop[c] + coa[c] + nad[c] --> co2[c] + ivcoa[c...,12.951498,14.030789,66.279632,62.578616
743,ICDHy,isocitrate dehydrogenase NADP,icit[c] + nadp[c] --> akg[c] + co2[c] + nadph[c],10.356372,11.219403,0.000000,0.000000
827,DPMVD,diphosphomevalonate decarboxylase,5dpmev[c] + atp[c] --> adp[c] + co2[c] + ipdp[...,2.617932,2.836093,67.053701,70.754717
209,FAS100COA,fatty acyl CoA synthase n C100CoA,3.0 h[c] + malcoa[c] + 2.0 nadph[c] + occoa[c]...,2.512921,2.722331,0.000000,0.000000
234,FAS80COA_L,fatty acyl CoA synthase n C80CoA lumped reaction,accoa[c] + 9.0 h[c] + 3.0 malcoa[c] + 6.0 nadp...,2.512921,2.722331,0.000000,0.000000


In [52]:
get_reactions_with_metabolite(fba_results_df, 'co2[m]')

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,glucose_max_fpp_flux,mannitol_max_fpp_flux
999,CO2tm,CO2 transport diffusion mitochondrial,co2[c] <=> co2[m],-108.680635,-117.737355,-197.677794,-175.471698
363,PDHm,pyruvate dehydrogenase,coa[m] + nad[m] + pyr[m] --> accoa[m] + co2[m]...,49.392433,53.508469,66.279632,62.578616
171,ICDHym,Isocitrate dehydrogenase NADP,icit[m] + nadp[m] --> akg[m] + co2[m] + nadph[m],19.719785,21.363101,0.000000,0.000000
107,AKGDam,oxoglutarate dehydrogenase lipoamide,akg[m] + h[m] + lpam[m] <=> co2[m] + sdhlam[m],17.598475,19.065015,0.000000,0.000000
1135,ACLSm,acetolactate synthase mitochondrial,h[m] + 2.0 pyr[m] --> alac_S[m] + co2[m],17.138303,18.566495,66.279632,62.578616
912,OXAGm,non enzymatic reaction,h[m] + oxag[m] <=> 2oxoadp[m] + co2[m],3.134551,3.395764,0.000000,0.000000
1123,ACHBSm,2 aceto 2 hydroxybutanoate synthase mitochondrial,2obut[m] + h[m] + pyr[m] --> 2ahbut[m] + co2[m],1.032289,1.118313,0.000000,0.000000
750,PSERDm_SC,phosphatidylserine decarboxylase yeast specifi...,h[m] + 0.01 ps_SC[m] --> co2[m] + 0.01 pe_SC[m],0.664798,0.720198,0.000000,0.000000
815,ME2m,malic enzyme NADP mitochondrial,mal_L[m] + nadp[m] --> co2[m] + nadph[m] + pyr[m],0.000000,0.000000,63.957426,38.050314
662,GLYCLm,glycine cleavage system mitochondrial,gly[m] + nad[m] + thf[m] --> co2[m] + mlthf[m]...,0.000000,0.000000,1.161103,12.264151


In [21]:
# isolate isocitrate dehydrogenase
fba_results_df[fba_results_df['reaction_id'].str.contains('ICDHy') == True]

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,glucose_max_fpp_flux,mannitol_max_fpp_flux
171,ICDHym,Isocitrate dehydrogenase NADP,icit[m] + nadp[m] --> akg[m] + co2[m] + nadph[m],19.719785,21.363101,0.000000,0.000000
743,ICDHy,isocitrate dehydrogenase NADP,icit[c] + nadp[c] --> akg[c] + co2[c] + nadph[c],10.356372,11.219403,0.000000,0.000000
745,ICDHyp,Isocitrate dehydrogenase NADP,icit[x] + nadp[x] --> akg[x] + co2[x] + nadph[x],0.000000,0.000000,0.000000,0.000000


In [23]:
model.reactions.get_by_id('ICDHy').metabolites

{<Metabolite akg[c] at 0x142b132b0>: 1.0,
 <Metabolite co2[c] at 0x142b129e0>: 1.0,
 <Metabolite icit[c] at 0x142b115d0>: -1.0,
 <Metabolite nadp[c] at 0x142b13a60>: -1.0,
 <Metabolite nadph[c] at 0x142b13ac0>: 1.0}

In [25]:
any(met.id == 'co2[c]' for met in model.reactions.get_by_id('ICDHy').metabolites)

True

### Identify reactions with differential flux values

In [14]:
# split the dataframe into two dataframes, one for biomass and one for fpp
biomass_df = fba_results_df[['reaction_id', 'reaction_name', 'full_reaction', 'glucose_max_biomass_flux', 'mannitol_max_biomass_flux']]
fpp_df = fba_results_df[['reaction_id', 'reaction_name', 'full_reaction', 'glucose_max_fpp_flux', 'mannitol_max_fpp_flux']]

display(biomass_df.head())
display(fpp_df.head())


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307,-608.649499
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652,370.077123
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108,363.432451
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474,353.744597
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,402.370631


,reaction_id,reaction_name,full_reaction,glucose_max_fpp_flux,mannitol_max_fpp_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-541.848089,-623.270440
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],281.761006,326.100629
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],281.761006,326.100629
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,281.761006,326.100629
421,EX_h2o(e),H2O exchange,h2o[e] <=>,354.136430,440.566038


### Add columns comparing the glucose and mannitol solutions

In [15]:
# a function that adds a columns for difference and percent difference
def add_difference_columns(_df, column_1, column_2):
    df = _df.copy()

    df['difference'] = df[column_1] - df[column_2]
    df['percent_difference'] = df['difference'] / df[column_1] * 100

    # add a column for the absolute value of the percent difference
    df['absolute_difference'] = abs(df['difference'])


    df = df.sort_values(by=['absolute_difference'], ascending=False)

    # drop the absolute percent difference column
    df = df.drop(columns=['absolute_difference'])

    return df

# add the difference columns to the dataframes
biomass_df = add_difference_columns(biomass_df, 'glucose_max_biomass_flux', 'mannitol_max_biomass_flux')
fpp_df = add_difference_columns(fpp_df, 'glucose_max_fpp_flux', 'mannitol_max_fpp_flux')

display(biomass_df.head())
display(fpp_df.head())

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,difference,percent_difference
360,HEX7,hexokinase D fructoseATP,atp[c] + fru[c] --> adp[c] + f6p[c] + h[c],0.000000,100.000000,-100.000000,-inf
1353,MNLDH2,mannitol dehydrogenase,mannitol[c] + nadp[c] --> fru[c] + h[c] + nadp...,0.000000,100.000000,-100.000000,-inf
413,EX_glc(e),D Glucose exchange,glc_D[e] <=>,-100.000000,0.000000,-100.000000,100.000000
1351,EX_mannitol_e,mannitol exchange,mannitol[e] -->,0.000000,-100.000000,100.000000,inf
1352,mannitol_transport,mannitol transport,mannitol[e] --> mannitol[c],0.000000,100.000000,-100.000000,-inf


,reaction_id,reaction_name,full_reaction,glucose_max_fpp_flux,mannitol_max_fpp_flux,difference,percent_difference
360,HEX7,hexokinase D fructoseATP,atp[c] + fru[c] --> adp[c] + f6p[c] + h[c],0.000000,100.000000,-100.000000,-inf
1353,MNLDH2,mannitol dehydrogenase,mannitol[c] + nadp[c] --> fru[c] + h[c] + nadp...,0.000000,100.000000,-100.000000,-inf
413,EX_glc(e),D Glucose exchange,glc_D[e] <=>,-100.000000,0.000000,-100.000000,100.000000
1351,EX_mannitol_e,mannitol exchange,mannitol[e] -->,0.000000,-100.000000,100.000000,inf
1352,mannitol_transport,mannitol transport,mannitol[e] --> mannitol[c],0.000000,100.000000,-100.000000,-inf


### 

### Glucose vs. mannitol (differential fluxes for max biomass)

In [16]:
# filter the biomass dataframe to only include reactions with greater than 5 flux units for at least one of the carbon sources
biomass_df = biomass_df[(abs(biomass_df['glucose_max_biomass_flux']) > 5) | (abs(biomass_df['mannitol_max_biomass_flux']) > 5)]

biomass_df.head(30)

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,difference,percent_difference
360,HEX7,hexokinase D fructoseATP,atp[c] + fru[c] --> adp[c] + f6p[c] + h[c],0.000000,100.000000,-100.000000,-inf
1353,MNLDH2,mannitol dehydrogenase,mannitol[c] + nadp[c] --> fru[c] + h[c] + nadp...,0.000000,100.000000,-100.000000,-inf
413,EX_glc(e),D Glucose exchange,glc_D[e] <=>,-100.000000,0.000000,-100.000000,100.000000
1351,EX_mannitol_e,mannitol exchange,mannitol[e] -->,0.000000,-100.000000,100.000000,inf
1352,mannitol_transport,mannitol transport,mannitol[e] --> mannitol[c],0.000000,100.000000,-100.000000,-inf
958,GLCt1,glucose transport uniport,glc_D[e] --> glc_D[c],100.000000,0.000000,100.000000,100.000000
711,HEX1,hexokinase D glucoseATP,atp[c] + glc_D[c] --> adp[c] + g6p[c] + h[c],100.000000,0.000000,100.000000,100.000000
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,402.370631,-77.105433,-23.705405
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-325.265198,-402.370631,77.105433,-23.705405
547,PGI,glucose 6 phosphate isomerase,g6p[c] <=> f6p[c],35.574485,-19.794308,55.368793,155.641870


### Glucose vs. mannitol (differential fluxes for max FPP)

In [17]:
# filter the fpp dataframe to only include reactions with greater than 5 flux units for at least one of the carbon sources
fpp_df = fpp_df[(abs(fpp_df['glucose_max_fpp_flux']) > 5) | (abs(fpp_df['mannitol_max_fpp_flux']) > 5)]

fpp_df.head(30)

,reaction_id,reaction_name,full_reaction,glucose_max_fpp_flux,mannitol_max_fpp_flux,difference,percent_difference
360,HEX7,hexokinase D fructoseATP,atp[c] + fru[c] --> adp[c] + f6p[c] + h[c],0.000000,100.000000,-100.000000,-inf
1353,MNLDH2,mannitol dehydrogenase,mannitol[c] + nadp[c] --> fru[c] + h[c] + nadp...,0.000000,100.000000,-100.000000,-inf
413,EX_glc(e),D Glucose exchange,glc_D[e] <=>,-100.000000,0.000000,-100.000000,100.000000
1351,EX_mannitol_e,mannitol exchange,mannitol[e] -->,0.000000,-100.000000,100.000000,inf
1352,mannitol_transport,mannitol transport,mannitol[e] --> mannitol[c],0.000000,100.000000,-100.000000,-inf
547,PGI,glucose 6 phosphate isomerase,g6p[c] <=> f6p[c],100.000000,0.000000,100.000000,100.000000
958,GLCt1,glucose transport uniport,glc_D[e] --> glc_D[c],100.000000,0.000000,100.000000,100.000000
711,HEX1,hexokinase D glucoseATP,atp[c] + glc_D[c] --> adp[c] + g6p[c] + h[c],100.000000,-0.000000,100.000000,100.000000
421,EX_h2o(e),H2O exchange,h2o[e] <=>,354.136430,440.566038,-86.429608,-24.405738
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-354.136430,-440.566038,86.429608,-24.405738


### Print out details of mevalonate pathway
The purpose of this was to determine the reason why carotenoids require NADPH.
<br><br>
The answer was the reaction of HMG-CoA + 2 NADPH -> Mevalonate (HMG-CoA reductase)

In [18]:
def print_metablite_details(metabolite_id):
    metabolite = model.metabolites.get_by_id(metabolite_id)
    print(f'{metabolite.name} ({metabolite.id}) reactions:')
    for reaction in metabolite.reactions:
        print(reaction, reaction.name)

    print()

print_metablite_details('accoa[c]')
print_metablite_details('hmgcoa[c]')
print_metablite_details('mev_R[c]')
print_metablite_details('5pmev[c]')
print_metablite_details('5dpmev[c]')
print_metablite_details('ipdp[c]')
print_metablite_details('dmpp[c]')
print_metablite_details('grdp[c]')
print_metablite_details('frdp[c]')

Acetyl_CoA (accoa[c]) reactions:
SERATi: accoa[c] + ser_L[c] --> acser[c] + coa[c] serine O acetyltransferase irreversible
IPPS: 3mob[c] + accoa[c] + h2o[c] --> 3c3hmp[c] + coa[c] + h[c] 2 isopropylmalate synthase
ACACCT: acac[c] + accoa[c] --> aacoa[c] + ac[c] Acetyl-CoA:acetoacetyl-CoA transferase
SPMDAT1: accoa[c] + spmd[c] --> N1aspmd[c] + coa[c] + h[c] Spermidine acetyltransferase
ACCOACr: accoa[c] + atp[c] + hco3[c] <=> adp[c] + h[c] + malcoa[c] + pi[c] acetyl CoA carboxylase reversible reaction
FAS80_L: accoa[c] + 8.0 h[c] + 3.0 malcoa[c] + 6.0 nadph[c] --> 3.0 co2[c] + 4.0 coa[c] + 2.0 h2o[c] + 6.0 nadp[c] + octa[c] fatty acid synthase n C80 lumped reaction
ATPCitL: atp[c] + cit[c] + coa[c] --> accoa[c] + adp[c] + oaa[c] + pi[c] ATP Citrate Lyase YL
CSNAT: accoa[c] + crn[c] --> acrn[c] + coa[c] carnitine O acetyltransferase
ACGAM6PS: accoa[c] + gam6p[c] <=> acgam6p[c] + coa[c] + h[c] N acetylglucosamine 6 phosphate synthase
HSERTA: accoa[c] + hom_L[c] <=> achms[c] + coa[c] homo